# MarketMinds - Complete Pipeline

**Run this entire notebook on Google Colab with GPU enabled.**

### Setup Instructions:
1. Go to Runtime → Change runtime type → Select **T4 GPU**
2. Click **Runtime → Run all**
3. Wait ~30-45 minutes
4. Download the `reports/` folder and `data/features/` when done

This notebook runs the entire pipeline:
- Data preparation (reshape, fetch prices, align, clean)
- Baseline models (Market-only, TF-IDF, Fused) with LogReg, SVM, XGBoost
- FinBERT embeddings extraction (GPU)
- FinBERT fused models with LogReg, SVM, XGBoost
- SHAP interpretability + ablation study
- Final comparison table

## 0. Setup Environment

In [1]:
# Install required packages
!pip install -q pandas numpy pyarrow yfinance torch transformers scikit-learn xgboost shap tqdm matplotlib

In [2]:
# Clone the repository (comment out if you uploaded files manually)
!git clone https://github.com/VarunTogaru/MarketMindsCS4774.git
%cd MarketMindsCS4774

Cloning into 'MarketMindsCS4774'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 97 (delta 46), reused 81 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 3.77 MiB | 11.36 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/MarketMindsCS4774


In [3]:
import os
import re
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModel

from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, brier_score_loss
import xgboost as xgb
import shap

print("Imports complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Imports complete!
PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [4]:
# Create output directories
os.makedirs("data/processed", exist_ok=True)
os.makedirs("data/features", exist_ok=True)
os.makedirs("reports", exist_ok=True)
print("Directories created!")

Directories created!


## 1. Data Preparation

In [5]:
# === STEP 1: Reshape headlines from wide to long format ===
print("Step 1: Reshaping headlines...")

df_raw = pd.read_csv("data/raw/DailyNews/Combined_News_DJIA.csv")
df_raw = df_raw.drop(columns=["Label"])  # We'll compute our own labels

headline_cols = [c for c in df_raw.columns if c.startswith("Top")]

long_df = df_raw.melt(
    id_vars=["Date"],
    value_vars=headline_cols,
    var_name="headline_id",
    value_name="headline_text"
)

long_df = long_df.dropna(subset=["headline_text"])
long_df["ticker"] = "DJIA"
long_df.to_parquet("data/raw/headlines_long.parquet", index=False)

print(f"  Headlines reshaped: {len(long_df):,} rows")

Step 1: Reshaping headlines...
  Headlines reshaped: 49,718 rows


In [6]:
# === STEP 2: Fetch DJIA prices from Yahoo Finance ===
print("Step 2: Fetching DJIA prices from Yahoo Finance...")

import yfinance as yf

djia = yf.download("^DJI", start="2008-08-01", end="2017-01-01", progress=False)
djia.reset_index(inplace=True)

# Handle multi-level columns if present
if isinstance(djia.columns, pd.MultiIndex):
    djia.columns = [col[0] if col[1] == '' else col[0] for col in djia.columns]

djia = djia.rename(columns={"Date": "trading_date", "Close": "close"})
djia = djia[["trading_date", "close", "Volume"]]
djia.to_parquet("data/raw/djia_prices.parquet", index=False)

print(f"  Prices fetched: {len(djia):,} trading days")

Step 2: Fetching DJIA prices from Yahoo Finance...
  Prices fetched: 2,120 trading days


In [7]:
# === STEP 3: Align headlines with prices and create labels ===
print("Step 3: Aligning headlines with prices and creating labels...")

headlines = pd.read_parquet("data/raw/headlines_long.parquet")
prices = pd.read_parquet("data/raw/djia_prices.parquet")

headlines["trading_date"] = pd.to_datetime(headlines["Date"])
prices["trading_date"] = pd.to_datetime(prices["trading_date"])

df = headlines.merge(prices, on="trading_date", how="inner")

prices = prices.sort_values("trading_date")
prices["close_t_plus_1"] = prices["close"].shift(-1)

df = df.merge(prices[["trading_date", "close_t_plus_1"]], on="trading_date", how="left")
df["return_t_plus_1"] = (df["close_t_plus_1"] - df["close"]) / df["close"]
df = df.dropna(subset=["return_t_plus_1"])
df["label"] = (df["return_t_plus_1"] > 0).astype(int)

df[["ticker", "trading_date", "headline_text", "return_t_plus_1", "label"]].to_parquet(
    "data/processed/model_table.parquet", index=False
)

print(f"  Aligned data: {len(df):,} headline-day pairs")

Step 3: Aligning headlines with prices and creating labels...
  Aligned data: 49,718 headline-day pairs


In [8]:
# === STEP 4: Clean text ===
print("Step 4: Cleaning headlines...")

# Text cleaning function
_BOILERPLATE_PREFIXES = [
    r"^breaking:\s*", r"^update\s*\d*:\s*", r"^exclusive:\s*",
    r"^reuters:\s*", r"^wsj:\s*", r"^cnbc:\s*", r"^bloomberg:\s*",
]

def clean_headline(text):
    if text is None:
        return ""
    s = str(text).strip().lower()
    for pat in _BOILERPLATE_PREFIXES:
        s = re.sub(pat, "", s)
    s = re.sub(r"http\S+|www\.\S+", "", s)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df = pd.read_parquet("data/processed/model_table.parquet")
df["clean_headline"] = df["headline_text"].apply(clean_headline)

before = len(df)
df = df[df["clean_headline"].str.len() > 0].copy()
df = df.drop_duplicates(subset=["trading_date", "clean_headline"]).copy()
after = len(df)

df.to_parquet("data/processed/model_table_clean.parquet", index=False)

print(f"  Cleaned data: {after:,} rows (dropped {before - after:,})")
print("\n" + "="*50)
print("DATA PREPARATION COMPLETE!")
print("="*50)

Step 4: Cleaning headlines...
  Cleaned data: 49,697 rows (dropped 21)

DATA PREPARATION COMPLETE!


## 2. Helper Functions

In [9]:
# Rolling time-series split (prevents lookahead bias)
def rolling_time_splits(df, date_col, train_days=365*4, test_days=365, step_days=90):
    """Generate rolling train/test date boundaries."""
    dates = pd.to_datetime(df[date_col])
    start = dates.min().normalize()
    end = dates.max().normalize()

    train_start = start
    train_end = train_start + pd.Timedelta(days=train_days)
    test_start = train_end
    test_end = test_start + pd.Timedelta(days=test_days)

    while test_end <= end:
        yield train_start, train_end, test_start, test_end
        train_start += pd.Timedelta(days=step_days)
        train_end = train_start + pd.Timedelta(days=train_days)
        test_start = train_end
        test_end = test_start + pd.Timedelta(days=test_days)

def evaluate_model(y_true, y_pred, y_prob):
    """Calculate all metrics including Brier score."""
    return {
        "auc": roc_auc_score(y_true, y_prob),
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "brier": brier_score_loss(y_true, y_prob),
    }

def get_models():
    """Return dict of model name -> model instance."""
    return {
        "LogisticRegression": LogisticRegression(max_iter=2000, class_weight="balanced"),
        "LinearSVM": CalibratedClassifierCV(LinearSVC(max_iter=2000, class_weight="balanced")),
        "XGBoost": xgb.XGBClassifier(
            n_estimators=100, max_depth=3, learning_rate=0.1,
            use_label_encoder=False, eval_metric='logloss', verbosity=0
        ),
    }

print("Helper functions defined!")

Helper functions defined!


## 3. Baseline Models (No FinBERT)

In [10]:
# Load cleaned data
df = pd.read_parquet("data/processed/model_table_clean.parquet").copy()
df["trading_date"] = pd.to_datetime(df["trading_date"])
df = df.sort_values("trading_date")

# Create day-level aggregation
day_df = (
    df.groupby("trading_date")
    .agg(
        doc=("clean_headline", lambda x: " ".join(x.tolist())),
        return_t_plus_1=("return_t_plus_1", "first")
    )
    .reset_index()
)
day_df["label"] = (day_df["return_t_plus_1"] > 0).astype(int)

# Add market features
day_df = day_df.sort_values("trading_date").copy()
day_df["ret_t"] = day_df["return_t_plus_1"].shift(1)
day_df["vol_5"] = day_df["ret_t"].rolling(5).std()
day_df["mom_5"] = day_df["ret_t"].rolling(5).mean()
day_df = day_df.dropna().copy()

market_cols = ["ret_t", "vol_5", "mom_5"]
print(f"Day-level dataset: {len(day_df)} trading days")

Day-level dataset: 1984 trading days


In [11]:
# === BASELINE 1: Market-Only Models ===
print("\n" + "="*50)
print("Running MARKET-ONLY baselines...")
print("="*50)

all_results = []

for model_name, model in get_models().items():
    print(f"  Training {model_name}...")

    for train_start, train_end, test_start, test_end in rolling_time_splits(day_df, "trading_date"):
        train_mask = (day_df["trading_date"] >= train_start) & (day_df["trading_date"] < train_end)
        test_mask = (day_df["trading_date"] >= test_start) & (day_df["trading_date"] < test_end)

        train_df = day_df.loc[train_mask]
        test_df = day_df.loc[test_mask]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_df[market_cols].values)
        X_test = scaler.transform(test_df[market_cols].values)
        y_train = train_df["label"].values
        y_test = test_df["label"].values

        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)

        metrics = evaluate_model(y_test, y_pred, y_prob)
        metrics.update({
            "feature_set": "market_only",
            "model": model_name,
            "train_start": str(train_start.date()),
            "test_start": str(test_start.date()),
            "n_train": len(train_df),
            "n_test": len(test_df),
        })
        all_results.append(metrics)

print("  Market-only baselines complete!")


Running MARKET-ONLY baselines...
  Training LogisticRegression...
  Training LinearSVM...
  Training XGBoost...
  Market-only baselines complete!


In [12]:
# === BASELINE 2: TF-IDF Text-Only Models ===
print("\n" + "="*50)
print("Running TF-IDF TEXT-ONLY baselines...")
print("="*50)

for model_name, model in get_models().items():
    print(f"  Training {model_name}...")

    for train_start, train_end, test_start, test_end in rolling_time_splits(day_df, "trading_date"):
        train_mask = (day_df["trading_date"] >= train_start) & (day_df["trading_date"] < train_end)
        test_mask = (day_df["trading_date"] >= test_start) & (day_df["trading_date"] < test_end)

        train_df = day_df.loc[train_mask]
        test_df = day_df.loc[test_mask]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=2)
        X_train = tfidf.fit_transform(train_df["doc"].values)
        X_test = tfidf.transform(test_df["doc"].values)
        y_train = train_df["label"].values
        y_test = test_df["label"].values

        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)

        metrics = evaluate_model(y_test, y_pred, y_prob)
        metrics.update({
            "feature_set": "tfidf_text_only",
            "model": model_name,
            "train_start": str(train_start.date()),
            "test_start": str(test_start.date()),
            "n_train": len(train_df),
            "n_test": len(test_df),
        })
        all_results.append(metrics)

print("  TF-IDF text-only baselines complete!")


Running TF-IDF TEXT-ONLY baselines...
  Training LogisticRegression...
  Training LinearSVM...
  Training XGBoost...
  TF-IDF text-only baselines complete!


In [13]:
# === BASELINE 3: TF-IDF + Market Fused Models ===
print("\n" + "="*50)
print("Running TF-IDF + MARKET FUSED baselines...")
print("="*50)

for model_name, model in get_models().items():
    print(f"  Training {model_name}...")

    for train_start, train_end, test_start, test_end in rolling_time_splits(day_df, "trading_date"):
        train_mask = (day_df["trading_date"] >= train_start) & (day_df["trading_date"] < train_end)
        test_mask = (day_df["trading_date"] >= test_start) & (day_df["trading_date"] < test_end)

        train_df = day_df.loc[train_mask]
        test_df = day_df.loc[test_mask]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        # TF-IDF features
        tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 2), min_df=2)
        X_text_train = tfidf.fit_transform(train_df["doc"].values)
        X_text_test = tfidf.transform(test_df["doc"].values)

        # Market features
        scaler = StandardScaler()
        X_mkt_train = scaler.fit_transform(train_df[market_cols].values)
        X_mkt_test = scaler.transform(test_df[market_cols].values)

        # Combine
        X_train = hstack([X_text_train, X_mkt_train])
        X_test = hstack([X_text_test, X_mkt_test])
        y_train = train_df["label"].values
        y_test = test_df["label"].values

        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)

        metrics = evaluate_model(y_test, y_pred, y_prob)
        metrics.update({
            "feature_set": "tfidf_market_fused",
            "model": model_name,
            "train_start": str(train_start.date()),
            "test_start": str(test_start.date()),
            "n_train": len(train_df),
            "n_test": len(test_df),
        })
        all_results.append(metrics)

print("  TF-IDF + Market fused baselines complete!")
print("\n" + "="*50)
print("ALL BASELINE MODELS COMPLETE!")
print("="*50)


Running TF-IDF + MARKET FUSED baselines...
  Training LogisticRegression...
  Training LinearSVM...
  Training XGBoost...
  TF-IDF + Market fused baselines complete!

ALL BASELINE MODELS COMPLETE!


## 4. FinBERT Embeddings (GPU)

In [14]:
print("\n" + "="*50)
print("Extracting FinBERT embeddings (this takes ~15-20 min with GPU)...")
print("="*50)

# Load FinBERT
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)

base_model.eval()
for p in base_model.parameters():
    p.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

print(f"Device: {device}")
print(f"Days to embed: {len(day_df)}")


Extracting FinBERT embeddings (this takes ~15-20 min with GPU)...


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device: cuda
Days to embed: 1984


In [15]:
def embed_texts(texts, max_length=128, batch_size=32):
    """Extract FinBERT embeddings using mean pooling."""
    all_vecs = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
            batch = texts[i:i+batch_size]

            enc = tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            ).to(device)

            out = base_model(**enc)
            hidden = out.last_hidden_state

            # Mean pooling
            mask = enc["attention_mask"].unsqueeze(-1)
            masked_hidden = hidden * mask
            sum_hidden = masked_hidden.sum(dim=1)
            denom = mask.sum(dim=1).clamp(min=1e-9)
            mean_pooled = sum_hidden / denom

            all_vecs.append(mean_pooled.cpu().numpy())

    return np.vstack(all_vecs)

# Extract embeddings
embeddings = embed_texts(day_df["doc"].tolist(), max_length=128, batch_size=32)
print(f"Embeddings shape: {embeddings.shape}")

Embedding:   0%|          | 0/62 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Embedding: 100%|██████████| 62/62 [00:15<00:00,  3.91it/s]

Embeddings shape: (1984, 768)


In [16]:
# Save embeddings
emb_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(embeddings.shape[1])])
emb_df.insert(0, "trading_date", day_df["trading_date"].values)
emb_df.insert(1, "label", day_df["label"].values)
emb_df.insert(2, "return_t_plus_1", day_df["return_t_plus_1"].values)

emb_df.to_parquet("data/features/finbert_day_embeddings.parquet", index=False)
print("Saved: data/features/finbert_day_embeddings.parquet")
print("\n" + "="*50)
print("FINBERT EMBEDDINGS COMPLETE!")
print("="*50)

Saved: data/features/finbert_day_embeddings.parquet

FINBERT EMBEDDINGS COMPLETE!


## 5. FinBERT Models

In [17]:
# Load embeddings and rebuild market features
emb = pd.read_parquet("data/features/finbert_day_embeddings.parquet").copy()
emb["trading_date"] = pd.to_datetime(emb["trading_date"])
emb = emb.sort_values("trading_date")

emb["ret_t"] = emb["return_t_plus_1"].shift(1)
emb["vol_5"] = emb["ret_t"].rolling(5).std()
emb["mom_5"] = emb["ret_t"].rolling(5).mean()
emb = emb.dropna().copy()

embedding_cols = [c for c in emb.columns if c.startswith("emb_")]
print(f"Embedding columns: {len(embedding_cols)}")
print(f"Days: {len(emb)}")

Embedding columns: 768
Days: 1979


In [18]:
# === FINBERT 1: FinBERT Text-Only Models ===
print("\n" + "="*50)
print("Running FinBERT TEXT-ONLY models...")
print("="*50)

for model_name, model in get_models().items():
    print(f"  Training {model_name}...")

    for train_start, train_end, test_start, test_end in rolling_time_splits(emb, "trading_date"):
        train_mask = (emb["trading_date"] >= train_start) & (emb["trading_date"] < train_end)
        test_mask = (emb["trading_date"] >= test_start) & (emb["trading_date"] < test_end)

        train_df = emb.loc[train_mask]
        test_df = emb.loc[test_mask]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        X_train = train_df[embedding_cols].values
        X_test = test_df[embedding_cols].values
        y_train = train_df["label"].values
        y_test = test_df["label"].values

        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)

        metrics = evaluate_model(y_test, y_pred, y_prob)
        metrics.update({
            "feature_set": "finbert_text_only",
            "model": model_name,
            "train_start": str(train_start.date()),
            "test_start": str(test_start.date()),
            "n_train": len(train_df),
            "n_test": len(test_df),
        })
        all_results.append(metrics)

print("  FinBERT text-only models complete!")


Running FinBERT TEXT-ONLY models...
  Training LogisticRegression...
  Training LinearSVM...
  Training XGBoost...
  FinBERT text-only models complete!


In [19]:
# === FINBERT 2: FinBERT + Market Fused Models ===
print("\n" + "="*50)
print("Running FinBERT + MARKET FUSED models...")
print("="*50)

for model_name, model in get_models().items():
    print(f"  Training {model_name}...")

    for train_start, train_end, test_start, test_end in rolling_time_splits(emb, "trading_date"):
        train_mask = (emb["trading_date"] >= train_start) & (emb["trading_date"] < train_end)
        test_mask = (emb["trading_date"] >= test_start) & (emb["trading_date"] < test_end)

        train_df = emb.loc[train_mask]
        test_df = emb.loc[test_mask]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        # FinBERT embeddings
        X_emb_train = train_df[embedding_cols].values
        X_emb_test = test_df[embedding_cols].values

        # Market features
        scaler = StandardScaler()
        X_mkt_train = scaler.fit_transform(train_df[market_cols].values)
        X_mkt_test = scaler.transform(test_df[market_cols].values)

        # Combine
        X_train = np.hstack([X_emb_train, X_mkt_train])
        X_test = np.hstack([X_emb_test, X_mkt_test])
        y_train = train_df["label"].values
        y_test = test_df["label"].values

        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)

        metrics = evaluate_model(y_test, y_pred, y_prob)
        metrics.update({
            "feature_set": "finbert_market_fused",
            "model": model_name,
            "train_start": str(train_start.date()),
            "test_start": str(test_start.date()),
            "n_train": len(train_df),
            "n_test": len(test_df),
        })
        all_results.append(metrics)

print("  FinBERT + Market fused models complete!")
print("\n" + "="*50)
print("ALL FINBERT MODELS COMPLETE!")
print("="*50)


Running FinBERT + MARKET FUSED models...
  Training LogisticRegression...
  Training LinearSVM...
  Training XGBoost...
  FinBERT + Market fused models complete!

ALL FINBERT MODELS COMPLETE!


## 6. Save All Results

In [20]:
# Save detailed results
results_df = pd.DataFrame(all_results)
results_df.to_csv("reports/all_model_results.csv", index=False)
print("Saved: reports/all_model_results.csv")
print(f"Total experiments: {len(results_df)}")

Saved: reports/all_model_results.csv
Total experiments: 180


In [21]:
# Create summary comparison table (averaged across folds)
summary = (
    results_df
    .groupby(["feature_set", "model"])
    [["auc", "accuracy", "precision", "recall", "brier"]]
    .mean()
    .round(4)
    .reset_index()
)

# Sort by AUC descending
summary = summary.sort_values("auc", ascending=False)
summary.to_csv("reports/model_comparison_summary.csv", index=False)
print("Saved: reports/model_comparison_summary.csv")
print("\n" + "="*50)
print("MODEL COMPARISON SUMMARY")
print("="*50)
print(summary.to_string(index=False))

Saved: reports/model_comparison_summary.csv

MODEL COMPARISON SUMMARY
         feature_set              model    auc  accuracy  precision  recall  brier
         market_only LogisticRegression 0.5330    0.5040     0.5628  0.3963 0.2502
  tfidf_market_fused LogisticRegression 0.5253    0.5143     0.5510  0.5620 0.2501
  tfidf_market_fused            XGBoost 0.5213    0.5295     0.5468  0.7363 0.2615
     tfidf_text_only            XGBoost 0.5158    0.5285     0.5459  0.7398 0.2626
     tfidf_text_only LogisticRegression 0.5150    0.5212     0.5514  0.6072 0.2499
         market_only            XGBoost 0.5031    0.5156     0.5408  0.6813 0.2613
   finbert_text_only            XGBoost 0.5024    0.5089     0.5374  0.6213 0.2698
         market_only          LinearSVM 0.4989    0.5381     0.5383  0.9993 0.2488
finbert_market_fused            XGBoost 0.4972    0.5053     0.5350  0.6066 0.2709
   finbert_text_only          LinearSVM 0.4945    0.5351     0.5352  0.9945 0.2491
     tfidf_text_o

## 7. Interpretability & Ablation

In [22]:
print("\n" + "="*50)
print("Running SHAP Interpretability Analysis...")
print("="*50)

# Use 80/20 time split for interpretability
cutoff = emb["trading_date"].quantile(0.8)
train_df = emb[emb["trading_date"] < cutoff].copy()
test_df = emb[emb["trading_date"] >= cutoff].copy()

# Prepare features
scaler = StandardScaler()
X_train_emb = train_df[embedding_cols].values
X_test_emb = test_df[embedding_cols].values
X_train_mkt = scaler.fit_transform(train_df[market_cols].values)
X_test_mkt = scaler.transform(test_df[market_cols].values)

X_train = np.hstack([X_train_emb, X_train_mkt])
X_test = np.hstack([X_test_emb, X_test_mkt])
y_train = train_df["label"].values
y_test = test_df["label"].values

# Train final model
clf = LogisticRegression(max_iter=2000, class_weight="balanced")
clf.fit(X_train, y_train)

# SHAP analysis
explainer = shap.LinearExplainer(clf, X_train, feature_perturbation="interventional")
shap_values = explainer.shap_values(X_test)

# Feature importance
feature_names = [f"emb_{i}" for i in range(len(embedding_cols))] + market_cols
mean_abs = np.mean(np.abs(shap_values), axis=0)
imp = pd.DataFrame({"feature": feature_names, "mean_abs_shap": mean_abs})
imp = imp.sort_values("mean_abs_shap", ascending=False)

imp.to_csv("reports/shap_feature_importance.csv", index=False)
print("Saved: reports/shap_feature_importance.csv")
print("\nTop 20 Most Important Features:")
print(imp.head(20).to_string(index=False))


Running SHAP Interpretability Analysis...
Saved: reports/shap_feature_importance.csv

Top 20 Most Important Features:
feature  mean_abs_shap
emb_412       0.187449
emb_742       0.173312
  emb_7       0.171276
emb_749       0.170527
emb_465       0.170313
emb_484       0.168632
emb_182       0.165442
emb_467       0.163695
emb_325       0.163490
emb_571       0.162138
emb_272       0.156947
emb_390       0.149784
emb_562       0.147870
emb_140       0.147497
emb_719       0.144668
emb_599       0.144086
 emb_45       0.138913
 emb_33       0.138339
emb_293       0.137599
  ret_t       0.135677


In [23]:
# Ablation Study
print("\n" + "="*50)
print("Running ABLATION STUDY...")
print("="*50)

def fit_eval(Xtr, ytr, Xte, yte):
    m = LogisticRegression(max_iter=2000, class_weight="balanced")
    m.fit(Xtr, ytr)
    p = m.predict_proba(Xte)[:, 1]
    pred = (p >= 0.5).astype(int)
    return {
        "auc": roc_auc_score(yte, p),
        "accuracy": accuracy_score(yte, pred),
        "brier": brier_score_loss(yte, p),
    }

# Three variants
ablation_results = []

# 1) FinBERT embeddings only
res = fit_eval(X_train_emb, y_train, X_test_emb, y_test)
res["model"] = "FinBERT only"
ablation_results.append(res)

# 2) Market only
res = fit_eval(X_train_mkt, y_train, X_test_mkt, y_test)
res["model"] = "Market only"
ablation_results.append(res)

# 3) Fused
res = fit_eval(X_train, y_train, X_test, y_test)
res["model"] = "Fused (FinBERT + Market)"
ablation_results.append(res)

ablation_df = pd.DataFrame(ablation_results)[["model", "auc", "accuracy", "brier"]]
ablation_df = ablation_df.sort_values("auc", ascending=False)
ablation_df.to_csv("reports/ablation_study.csv", index=False)

print("Saved: reports/ablation_study.csv")
print("\nAblation Results:")
print(ablation_df.to_string(index=False))


Running ABLATION STUDY...
Saved: reports/ablation_study.csv

Ablation Results:
                   model      auc  accuracy    brier
             Market only 0.518954  0.520202 0.250295
Fused (FinBERT + Market) 0.467832  0.500000 0.307302
            FinBERT only 0.467577  0.477273 0.305769


## 8. Final Summary

In [24]:
print("\n" + "="*70)
print("PIPELINE COMPLETE!")
print("="*70)

print("\nGenerated Files:")
print("-" * 40)

for folder in ["data/raw", "data/processed", "data/features", "reports"]:
    if os.path.exists(folder):
        files = os.listdir(folder)
        for f in files:
            path = os.path.join(folder, f)
            if os.path.isfile(path):
                size = os.path.getsize(path) / 1024
                print(f"  {path} ({size:.1f} KB)")

print("\n" + "="*70)
print("NEXT STEPS:")
print("="*70)
print("1. Download the 'reports/' folder (contains all metrics)")
print("2. Download 'data/features/finbert_day_embeddings.parquet'")
print("3. Download this notebook (File → Download → Download .ipynb)")
print("4. Upload everything to your GitHub repo")
print("\nKey files for your report:")
print("  - reports/model_comparison_summary.csv  (main results table)")
print("  - reports/ablation_study.csv            (text vs market contribution)")
print("  - reports/shap_feature_importance.csv   (interpretability)")


PIPELINE COMPLETE!

Generated Files:
----------------------------------------
  data/raw/headlines_long.parquet (4022.5 KB)
  data/raw/djia_prices.parquet (49.8 KB)
  data/processed/model_table_clean.parquet (7740.4 KB)
  data/processed/model_table.parquet (4116.1 KB)
  data/features/finbert_day_embeddings.parquet (8428.6 KB)
  reports/shap_feature_importance.csv (21.2 KB)
  reports/all_model_results.csv (26.5 KB)
  reports/model_comparison_summary.csv (1.0 KB)
  reports/ablation_study.csv (0.2 KB)

NEXT STEPS:
1. Download the 'reports/' folder (contains all metrics)
2. Download 'data/features/finbert_day_embeddings.parquet'
3. Download this notebook (File → Download → Download .ipynb)
4. Upload everything to your GitHub repo

Key files for your report:
  - reports/model_comparison_summary.csv  (main results table)
  - reports/ablation_study.csv            (text vs market contribution)
  - reports/shap_feature_importance.csv   (interpretability)
